In [199]:
!pip install azure-cognitiveservices-speech
!pip install librosa
!pip install pydub

In [200]:
import json
import os
import time
import azure.cognitiveservices.speech as speechsdk
import pandas as pd
import librosa
from pydub import AudioSegment
from pydub.silence import split_on_silence

In [201]:
SPEECH_KEY="d0c4c8a9daf64fd7bc4451ada99d8003"
SPEECH_REGION="eastus"

In [202]:
lista_directorio=[]
lista_directorio=!ls
lista_archivos=[]

for i in lista_directorio:
    if(i.endswith(".wav")==True):
        lista_archivos.append(i)

In [203]:
#aca se guardan los datos finales
dfFinal=pd.DataFrame()
lista_tiempos=[]

In [204]:
#Removedor de silencios
def remover_silencios (archivo):
    # Load the input audio file
    input_file = archivo
    audio = AudioSegment.from_wav(input_file)

    # Define silence settings (adjust these as needed)
    min_silence_len = 500  # Minimum silence duration in milliseconds
    silence_thresh = -40  # Silence threshold in dBFS

    # Split audio into non-silent chunks
    audio_chunks = split_on_silence(audio, silence_thresh=silence_thresh, min_silence_len=min_silence_len)

    # Create a new audio segment by concatenating non-silent chunks
    output_audio = AudioSegment.empty()
    for chunk in audio_chunks:
        output_audio += chunk

    # Save the cleaned audio to a new file
    output_file = archivo
    output_audio.export(output_file, format="wav")
    

In [205]:
#creamos variables globales para llevar al data frame

for j in lista_archivos:
    archivo_audio = j
    salida = []
    aux1 = {}
    dfAux = pd.DataFrame()
    lista_orden = []
    lista_aux=[]
    #######################################################
    #funcion para guardar tiempos de audios y remover silencios
    original, sr = librosa.load(j)
    tiempo_original=librosa.get_duration(y=original, sr=sr)
    remover_silencios(j)
    modificado, sr = librosa.load(j)
    tiempo_sin_silencios=librosa.get_duration(y=modificado, sr=sr)
    lista_aux=[round(tiempo_original), round(tiempo_sin_silencios), j]
    lista_tiempos.append(lista_aux)
    #######################################################
    
    def conversation_transcriber_recognition_canceled_cb(evt: speechsdk.SessionEventArgs):
        print('Canceled event')

    def conversation_transcriber_session_stopped_cb(evt: speechsdk.SessionEventArgs):
        print('SessionStopped event')

    def conversation_transcriber_transcribed_cb(evt: speechsdk.SpeechRecognitionEventArgs):
        #print('TRANSCRIBED:')
        if evt.result.reason == speechsdk.ResultReason.RecognizedSpeech:
            #print('\tText={}'.format(evt.result.text))
            aux1 = (json.dumps({"Text": "{}".format(evt.result.text), "Speaker": "{}".format(evt.result.speaker_id)}))
            salida.append(aux1)
            #print('\tSpeaker ID={}'.format(evt.result.speaker_id))
        elif evt.result.reason == speechsdk.ResultReason.NoMatch:
            print('\tNOMATCH: Speech could not be TRANSCRIBED: {}'.format(evt.result.no_match_details))

    def conversation_transcriber_session_started_cb(evt: speechsdk.SessionEventArgs):
        print('SessionStarted event')

    def recognize_from_file():

        speech_config = speechsdk.SpeechConfig(subscription=SPEECH_KEY, region=SPEECH_REGION)
        speech_config.speech_recognition_language="es-AR"

        audio_config = speechsdk.audio.AudioConfig(filename=archivo_audio)
        conversation_transcriber = speechsdk.transcription.ConversationTranscriber(speech_config=speech_config, audio_config=audio_config)
        transcribing_stop = False

        def stop_cb(evt: speechsdk.SessionEventArgs):
            #"""callback that signals to stop continuous recognition upon receiving an event `evt`"""
            print('CLOSING on {}'.format(evt))
            nonlocal transcribing_stop
            transcribing_stop = True

        # Connect callbacks to the events fired by the conversation transcriber
        conversation_transcriber.transcribed.connect(conversation_transcriber_transcribed_cb)
        conversation_transcriber.session_started.connect(conversation_transcriber_session_started_cb)
        conversation_transcriber.session_stopped.connect(conversation_transcriber_session_stopped_cb)
        conversation_transcriber.canceled.connect(conversation_transcriber_recognition_canceled_cb)
        # stop transcribing on either session stopped or canceled events
        conversation_transcriber.session_stopped.connect(stop_cb)
        conversation_transcriber.canceled.connect(stop_cb)

        conversation_transcriber.start_transcribing_async()

        # Waits for completion.
        while not transcribing_stop:
            time.sleep(.5)
        conversation_transcriber.stop_transcribing_async()    

    #ejecutamos la funcion que transcribe el texto
    recognize_from_file()
    
    #aca empezamos a manipular resultados
    for i in range(len(salida)):
        json_string = salida[i]
        json_object = json.loads(json_string)
        json_object["id_audio"]=archivo_audio
        json_object["orden"]=i
        dfAux=pd.json_normalize(json_object)
        dfFinal=pd.concat([dfFinal, dfAux])

SessionStarted event
Canceled event
CLOSING on ConversationTranscriptionCanceledEventArgs(session_id=2e2cc7e286b84bfca9849f17b293e5be, result=ConversationTranscriptionResult(result_id=4ff7c03f0690420ca24eee3c5d1e4c42, speaker_id=, text=, reason=ResultReason.Canceled))
SessionStopped event
CLOSING on SessionEventArgs(session_id=2e2cc7e286b84bfca9849f17b293e5be)
SessionStarted event
Canceled event
CLOSING on ConversationTranscriptionCanceledEventArgs(session_id=17ad9348928e4201b98534de90f1bdfc, result=ConversationTranscriptionResult(result_id=6e17b94c814e4d8398dfd5a08c470dcf, speaker_id=, text=, reason=ResultReason.Canceled))
SessionStopped event
CLOSING on SessionEventArgs(session_id=17ad9348928e4201b98534de90f1bdfc)
SessionStarted event
Canceled event
CLOSING on ConversationTranscriptionCanceledEventArgs(session_id=0bb02dc241b248c88feb9de5c55cd846, result=ConversationTranscriptionResult(result_id=ad0dd86ee6f04714abfb39c338fad8d6, speaker_id=, text=, reason=ResultReason.Canceled))
Sessi

In [206]:
dfFinal=dfFinal.reset_index(drop=True)

In [207]:
dfTiempos=pd.DataFrame(lista_tiempos, columns=["tiempo_original", "tiempo_modificado", "nombre_archivo"])
dfTiempos.tiempo_original=dfTiempos.tiempo_original.astype(int)
dfTiempos.tiempo_modificado=dfTiempos.tiempo_modificado.astype(int)
dfTiempos

,tiempo_original,tiempo_modificado,nombre_archivo
0,223,223,caso CAS-11558573-G8H5C1.wav
1,113,113,caso CAS-11560264-Y1D9M5.wav
2,77,77,caso CAS-11560748-B8S2G5.wav
3,61,61,Caso CAS-11567570-V0L9L8.wav
4,126,126,caso CAS-11569288-L9M3L3.wav


In [293]:
dfFinal=dfResultados.copy()
lista=[]
#tengo que recorrer todos los audios
for x in lista_archivos:
    df= pd.DataFrame()
    df=dfFinal.loc[dfFinal.id_audio==x].copy().reset_index(drop=True)

    # Creo una nueva columna 'Grouped_Text' que contiene los mensajes agrupados por Speaker
    df['Grouped_Text'] = (df['Speaker'] != df['Speaker'].shift(1)).cumsum()
    df = df.groupby(['Speaker', 'Grouped_Text'])['Text'].apply(' '.join).reset_index()
    
    #ordeno los mensajes agrupados
    df.sort_values(by=["Grouped_Text"], ascending =True)
    
    #le asigno el nombre del audio a estos mensajes agrupados
    df["id_audio"]=x
    df.rename(columns={"Grouped_Text":"Orden"}, inplace=True)
    
    #guardamos los resultados en una lista auxiliar
    lista.append(df.values.tolist())
    
#dejamos todo en un dataframe final
dfAux=pd.DataFrame()
dfFinal2=pd.DataFrame()
for y in range(len(lista)):
    for x in range(len(lista[y])):
        dfAux = pd.DataFrame(lista[y][x]).T
        dfFinal2=pd.concat([dfAux, dfFinal2])

In [294]:
dfFinal2.rename(columns={0:"Speaker", 1:"Orden", 2:"Texto", 3:"id_audio"}, inplace=True)

In [299]:
dfFinal2.to_csv("salida_prueba_resumida.csv", sep="|", encoding='utf-8-sig')